In [ ]:
!pip install wandb
import wandb
wandb.init(project="DL-1", entity="mshoaibvohra", settings=wandb.Settings(init_timeout=200))


CommError: Run initialization has timed out after 200.0 sec. Please try increasing the timeout with the `init_timeout` setting: `wandb.init(settings=wandb.Settings(init_timeout=120))`.

In [1]:
import os
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import torch
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split

# Predefined Taxonomies
taxonomy_first = {
    "URW": 0,  # War-related narratives
    "CC": 1,   # Climate change narratives
    "Other": 2  # Other narratives
}

taxonomy_URW = {
    "Blaming the war on others rather than the invader": [
        "Ukraine is the aggressor",
        "The West are the aggressors",
    ],
    "Discrediting Ukraine": [
        "Rewriting Ukraine’s history",
        "Discrediting Ukrainian nation and society",
        "Discrediting Ukrainian military",
        "Discrediting Ukrainian government and officials and policies",
        "Ukraine is a puppet of the West",
        "Ukraine is a hub for criminal activities",
        "Ukraine is associated with nazism",
        "Situation in Ukraine is hopeless",
    ],
    "Russia is the Victim": [
        "The West is russophobic",
        "Russia actions in Ukraine are only self-defence",
        "UA is anti-RU extremists",
    ],
    "Praise of Russia": [
        "Praise of Russian military might",
        "Praise of Russian President Vladimir Putin",
        "Russia is a guarantor of peace and prosperity",
        "Russia has international support from a number of countries and people",
        "Russian invasion has strong national support",
    ],
    "Overpraising the West": [
        "NATO will destroy Russia",
        "The West belongs in the right side of history",
        "The West has the strongest international support",
    ],
    "Speculating war outcomes": [
        "Russian army is collapsing",
        "Russian army will lose all the occupied territories",
        "Ukrainian army is collapsing",
    ],
    "Discrediting the West, Diplomacy": [
        "The EU is divided",
        "The West is weak",
        "The West is overreacting",
        "The West does not care about Ukraine, only about its interests",
        "Diplomacy does/will not work",
        "West is tired of Ukraine",
    ],
    "Negative Consequences for the West": [
        "Sanctions imposed by Western countries will backfire",
        "The conflict will increase the Ukrainian refugee flows to Europe",
    ],
    "Distrust towards Media": [
        "Western media is an instrument of propaganda",
        "Ukrainian media cannot be trusted",
    ],
    "Amplifying war-related fears": [
        "By continuing the war we risk WWIII",
        "Russia will also attack other countries",
        "There is a real possibility that nuclear weapons will be employed",
        "NATO should/will directly intervene",
    ],
    "Hidden plots by secret schemes of powerful groups": [],

}
taxonomy_CC = {"Criticism of climate policies": [
        "Climate policies are ineffective",
        "Climate policies have negative impact on the economy",
        "Climate policies are only for profit",
    ],
    "Criticism of institutions and authorities": [
        "Criticism of the EU",
        "Criticism of international entities",
        "Criticism of national governments",
        "Criticism of political organizations and figures",
    ],
    "Climate change is beneficial": [
        "CO2 is beneficial",
        "Temperature increase is beneficial",
    ],
    "Downplaying climate change": [
        "Climate cycles are natural",
        "Weather suggests the trend is global cooling",
        "Temperature increase does not have significant impact",
        "CO2 concentrations are too small to have an impact",
        "Human activities do not impact climate change",
        "Ice is not melting",
        "Sea levels are not rising",
        "Humans and nature will adapt to the changes",
    ],
    "Questioning the measurements and science": [
        "Methodologies/metrics used are unreliable/faulty",
        "Data shows no temperature increase",
        "Greenhouse effect/carbon dioxide do not drive climate change",
        "Scientific community is unreliable",
    ],
    "Criticism of climate movement": [
        "Climate movement is alarmist",
        "Climate movement is corrupt",
        "Ad hominem attacks on key activists",
    ],
    "Controversy about green technologies": [
        "Renewable energy is dangerous",
        "Renewable energy is unreliable",
        "Renewable energy is costly",
        "Nuclear energy is not climate friendly",
    ],
    "Amplifying Climate Fears": [
        "Earth will be uninhabitable soon",
        "Amplifying existing fears of global warming",
        "Doomsday scenarios for humans",
        "Whatever we do it is already too late",
    ],
    "Green policies are geopolitical instruments": [
        "Climate-related international relations are abusive/exploitative",
        "Green activities are a form of neo-colonialism",
    ],
               }
taxonomy_other = {"Other": []}

In [7]:
import os
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score

def parse_annotations(annotation_file):
    """Parse the annotations file and extract group, narrative, and subnarrative labels."""
    data = []
    with open(annotation_file, 'r') as f:
        for line in f:
            parts = line.strip().split('\t')
            if len(parts) >= 3:
                file_name = parts[0]
                group_narrative_sub = parts[2].split(";")

                groups = set()
                narratives = set()
                subnarratives_list = set()

                for entry in group_narrative_sub:
                    levels = entry.split(":")
                    if len(levels) >= 1:
                        groups.add(levels[0].strip())
                    if len(levels) >= 2:
                        narratives.add(levels[1].strip())
                    if len(levels) == 3:
                        subnarratives_list.add(levels[2].strip())

                data.append({
                    "file_name": file_name,
                    "groups": list(groups),
                    "narratives": list(narratives),
                    "subnarratives": list(subnarratives_list),
                })
    return data

def encode_labels(data, level_mapping):
    """Convert labels into one-hot encoded format."""
    labels = []
    for item in data:
        label_array = [0] * len(level_mapping)
        for label in item:
            if label in level_mapping:
                label_array[level_mapping[label]] = 1
        labels.append(label_array)
    return torch.tensor(labels, dtype=torch.float)

def load_and_encode_dataset(folder_path, level_mapping, level_key):
    """Load dataset and encode it for training."""
    annotation_file = os.path.join(folder_path, 'subtask-2-annotations.txt')
    data = parse_annotations(annotation_file)

    texts = []
    labels = []

    for item in data:
        file_path = os.path.join(folder_path, item['file_name'])
        if os.path.exists(file_path):
            with open(file_path, 'r') as f:
                texts.append(f.read())
                labels.append(item[level_key])

    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

    encodings = tokenizer(texts, truncation=True, padding=True, max_length=512, return_tensors="pt")
    label_tensor = encode_labels(labels, level_mapping)

    return encodings, label_tensor

class CustomDataset(torch.utils.data.Dataset):
    """Custom PyTorch dataset for loading text and labels."""
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)

def compute_metrics(p):
    """Compute evaluation metrics for model performance."""
    logits = p.predictions
    probabilities = torch.sigmoid(torch.tensor(logits)).numpy()
    preds = (probabilities >= 0.5).astype(int)
    labels = p.label_ids

    f1 = f1_score(labels, preds, average='micro')
    precision = precision_score(labels, preds, average='micro', zero_division=1)
    recall = recall_score(labels, preds, average='micro')

    return {'f1': f1, 'precision': precision, 'recall': recall}

def filter_by_group(encodings, labels, selected_groups):
    """Filter encodings and labels based on selected groups."""
    mask = selected_groups.any(axis=1)  # Create a mask based on group selection
    filtered_encodings = {key: val[mask] for key, val in encodings.items()}
    filtered_labels = labels[mask]  # Filter labels using the same mask
    return filtered_encodings, filtered_labels

def filter_by_narrative(encodings, labels, selected_narratives):
    """Filter encodings and labels based on selected narratives."""
    mask = selected_narratives.any(axis=1)  # Create a mask based on narrative selection
    filtered_encodings = {key: val[mask] for key, val in encodings.items()}
    filtered_labels = labels[mask]  # Filter labels using the same mask
    return filtered_encodings, filtered_labels

# Training Function
def train_model(encodings, labels, output_dir, num_labels):
    """Train a BERT model on the given encodings and labels."""
    train_inputs, val_inputs, train_labels, val_labels = train_test_split(
        encodings['input_ids'], labels, test_size=0.2, random_state=42
    )
    train_masks, val_masks = train_test_split(
        encodings['attention_mask'], test_size=0.2, random_state=42
    )

    train_encodings = {'input_ids': train_inputs, 'attention_mask': train_masks}
    val_encodings = {'input_ids': val_inputs, 'attention_mask': val_masks}

    train_dataset = CustomDataset(train_encodings, train_labels)
    val_dataset = CustomDataset(val_encodings, val_labels)

    model = BertForSequenceClassification.from_pretrained(
        'bert-base-uncased',
        num_labels=num_labels,
        problem_type="multi_label_classification"
    )

    training_args = TrainingArguments(
        output_dir=output_dir,
        evaluation_strategy="epoch",
        num_train_epochs=9,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        save_steps=500,
        save_total_limit=2,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=compute_metrics,
    )

    trainer.train()
    return trainer, model

In [16]:
def hierarchical_training_with_multiple_models(train_dir):
    print("Training Group-Level Classifier...")

    # Load and train the group-level classifier
    group_encodings, group_labels = load_and_encode_dataset(train_dir, taxonomy_first, 'groups')
    group_trainer, group_model = train_model(group_encodings, group_labels, './group_results', len(taxonomy_first))

    # Predict groups
    group_predictions = group_trainer.predict(CustomDataset(group_encodings, group_labels)).predictions
    selected_groups = (group_predictions >= 0.5).astype(int)

    # Define mappings for groups
    group_mapping = {idx: name for idx, name in enumerate(["URW", "CC"])}

    # Loop through each group (URW and CC)
    for group_idx, group_name in group_mapping.items():
        # Skip if the group wasn't selected in predictions
        if not selected_groups[:, group_idx].any():
            print(f"Skipping {group_name} since it wasn't selected.")
            continue

        print(f"Training Narrative-Level Classifier for {group_name}...")

        # Get the taxonomy for the current group
        taxonomy_group = taxonomy_URW if group_name == "URW" else taxonomy_CC

        # Train the narrative-level model for the current group
        narrative_encodings, narrative_labels = load_and_encode_dataset(
            train_dir,
            {key: idx for idx, key in enumerate(taxonomy_group.keys())},
            'narratives'
        )
        narrative_filtered_encodings, narrative_filtered_labels = filter_by_group(
            narrative_encodings, narrative_labels, selected_groups[:, group_idx].reshape(-1, 1)
        )
        narrative_trainer, narrative_model = train_model(
            narrative_filtered_encodings, narrative_filtered_labels, f'./narrative_results_{group_name}', len(taxonomy_group)
        )

        # Predict narratives for the current group
        narrative_predictions = narrative_trainer.predict(
            CustomDataset(narrative_encodings, narrative_labels)
        ).predictions
        selected_narratives = (narrative_predictions >= 0.5).astype(int)

        print(f"Training Subnarrative-Level Classifier for {group_name}...")

        # Create subnarrative mapping for the current group's taxonomy
        subnarratives_mapping = {
            sub: idx for narrative, subs in taxonomy_group.items() for idx, sub in enumerate(subs)
        }


        # Train the subnarrative-level model for the current group
        subnarrative_encodings, subnarrative_labels = load_and_encode_dataset(
            train_dir, subnarratives_mapping, 'subnarratives'
        )
        subnarrative_filtered_encodings, subnarrative_filtered_labels = filter_by_narrative(
            subnarrative_encodings, subnarrative_labels, selected_narratives
        )
        train_model(
            subnarrative_filtered_encodings, subnarrative_filtered_labels, f'./subnarrative_results_{group_name}', len(subnarratives_mapping)
        )

    print("All models trained successfully!")


In [8]:
hierarchical_training_with_multiple_models('train')

Training Group-Level Classifier...


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

Training Narrative-Level Classifier for CC...


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Training Subnarrative-Level Classifier for CC...


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


All models trained successfully!


In [9]:
def get_latest_checkpoint(folder):
    checkpoints = [d for d in os.listdir(folder) if d.startswith('checkpoint')]
    checkpoints.sort(key=lambda x: int(x.split('-')[-1]))
    return os.path.join(folder, checkpoints[-1])

In [25]:
def hierarchical_testing_with_dependency(test_dir, output_file):
    import os
    import torch
    from transformers import BertTokenizer, BertForSequenceClassification

    # Load trained group-level model
    group_model_path = get_latest_checkpoint('./group_results')
    group_model = BertForSequenceClassification.from_pretrained(group_model_path)
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

    # Dynamically generate group mapping based on model output size
    num_labels = group_model.config.num_labels
    group_mapping = {i: group for i, group in enumerate(["URW", "CC", "Other"][:num_labels])}

    results = []

    # Parse test files
    test_files = [f for f in os.listdir(test_dir) if os.path.isfile(os.path.join(test_dir, f))]

    for test_file in test_files:
        file_path = os.path.join(test_dir, test_file)

        with open(file_path, 'r') as f:
            text = f.read()

        # Tokenize the text
        encoding = tokenizer(text, truncation=True, padding=True, max_length=512, return_tensors="pt")

        # Step 1: Predict Group
        group_logits = group_model(**{key: val.to(group_model.device) for key, val in encoding.items()}).logits
        group_probs = torch.sigmoid(group_logits).cpu().detach().numpy()
        group_preds = (group_probs >= 0.5).astype(int)

        selected_groups = [
            group_mapping[idx] for idx, pred in enumerate(group_preds[0]) if pred == 1
        ]

        if not selected_groups or "Other" in selected_groups:
            # Skip to next file if "Other" is predicted
            results.append({
                "file": test_file,
                "narratives": ["Other"],
                "subnarratives": ["Other"]
            })
            continue

        selected_narratives = []
        selected_subnarratives = []

        # Step 2: Predict Narratives for each group
        for group_name in selected_groups:
            print(f"Predicting for group: {group_name}")

            # Load narrative model for the selected group
            narrative_model_path = get_latest_checkpoint(f'./narrative_results_{group_name}')
            narrative_model = BertForSequenceClassification.from_pretrained(narrative_model_path)

            taxonomy_group = taxonomy_URW if group_name == "URW" else taxonomy_CC
            narrative_logits = narrative_model(
                **{key: val.to(narrative_model.device) for key, val in encoding.items()}
            ).logits
            narrative_probs = torch.sigmoid(narrative_logits).cpu().detach().numpy()
            narrative_preds = (narrative_probs >= 0.5).astype(int)

            group_narratives = [
                narrative for idx, narrative in enumerate(taxonomy_group.keys()) if narrative_preds[0][idx] == 1
            ]

            selected_narratives.extend(group_narratives)

            # Step 3: Predict Subnarratives for the selected narratives
            subnarratives_mapping = {
                sub: idx for narrative, subs in taxonomy_group.items() for idx, sub in enumerate(subs)
            }
            subnarrative_model_path = get_latest_checkpoint(f'./subnarrative_results_{group_name}')
            subnarrative_model = BertForSequenceClassification.from_pretrained(subnarrative_model_path)

            subnarrative_logits = subnarrative_model(
                **{key: val.to(subnarrative_model.device) for key, val in encoding.items()}
            ).logits
            subnarrative_probs = torch.sigmoid(subnarrative_logits).cpu().detach().numpy()
            subnarrative_preds = (subnarrative_probs >= 0.5).astype(int)

            group_subnarratives = [
                sub for idx, sub in enumerate(subnarratives_mapping.keys()) if subnarrative_preds[0][idx] == 1
            ]

            selected_subnarratives.extend(group_subnarratives)

        # Collect results
        if not selected_narratives:
            selected_narratives = ["Other"]
        if not selected_subnarratives:
            selected_subnarratives = ["Other"]

        results.append({
            "file": test_file,
            "narratives": selected_narratives,
            "subnarratives": selected_subnarratives
        })

    # Save results to output file
    with open(output_file, 'w') as f:
        f.write("article_id\tnarratives\tsubnarratives\n")  # Write header
        for result in results:
            narratives = ";".join(result["narratives"])
            subnarratives = ";".join(result["subnarratives"])
            f.write(f"{result['file']}\t{narratives}\t{subnarratives}\n")

    # Print the output file content
    with open(output_file, 'r') as f:
        print(f.read())


In [26]:
hierarchical_testing_with_dependency('test', 'output_file.txt')

Predicting for group: URW
Predicting for group: CC
Predicting for group: CC
Predicting for group: CC
Predicting for group: CC
Predicting for group: URW
Predicting for group: URW
Predicting for group: CC
Predicting for group: URW
Predicting for group: CC
Predicting for group: URW
Predicting for group: CC
Predicting for group: URW
Predicting for group: CC
Predicting for group: URW
Predicting for group: URW
Predicting for group: URW
Predicting for group: URW
Predicting for group: CC
Predicting for group: URW
Predicting for group: CC
Predicting for group: CC
Predicting for group: CC
Predicting for group: CC
Predicting for group: URW
Predicting for group: URW
Predicting for group: CC
article_id	narratives	subnarratives
EN_UA_DEV_213.txt	Other	Other
EN_CC_200071.txt	Criticism of climate policies;Criticism of institutions and authorities	Climate policies are only for profit
EN_CC_200035.txt	Other	Other
EN_CC_200050.txt	Criticism of climate policies;Criticism of institutions and authorities	Ot